In [1]:
import requests

#import trex
import json
import pprint
from IPython.display import clear_output

import pickle

import pandas as pd
import numpy as np

from datasets import Dataset 

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

import evaluate

C:\Users\Administrator\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.empty_cache()
import gc
# del variables
gc.collect()
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# device = "cpu"
device


device(type='cuda', index=0)

In [4]:
# API request

def expand(text):
    #numberCandidates = 5
    response = requests.get("https://qanswer-core1.univ-st-etienne.fr/api/entitylinker", params={'text': text, 'language': 'en', 'knowledgebase': 'wikidata'})
    #print(response.json())

    input = text + " "
    input_entities = []
    for r in response.json():
        if 'uri' in r and 'http://www.wikidata.org/entity/' in r['uri']:
            input_entities.append("(" + r['text'] + " "+ r['uri'].replace('http://www.wikidata.org/entity/','') + ")") 
    return dict({'input': input, 'input_entities':input_entities})

expand("Rome is the capital of Italy")

{'input': 'Rome is the capital of Italy ',
 'input_entities': ['(rome Q220)',
  '(rome Q17167)',
  '(rome Q37571708)',
  '(rome Q37501701)',
  '(rome Q99672149)',
  '(rome Q6580)',
  '(rome Q209878)',
  '(rome Q6586)',
  '(rome Q2165048)',
  '(rome Q18287233)',
  '(rome Q1925709)',
  '(rome Q3448522)',
  '(rome Q567806)',
  '(rome Q952870)',
  '(rome Q1180902)',
  '(rome Q575847)',
  '(rome Q2673079)',
  '(rome Q3459993)',
  '(rome Q53575037)',
  '(rome Q3506769)',
  '(the capital Q3520197)',
  '(the capital Q106493293)',
  '(the capital Q17153073)',
  '(the capital Q63534436)',
  '(capital Q193893)',
  '(capital Q28026026)',
  '(capital Q204627)',
  '(capital Q6206)',
  '(capital Q8137)',
  '(capital Q58784)',
  '(capital Q98912)',
  '(capital Q51929311)',
  '(capital Q24008612)',
  '(capital Q5035441)',
  '(capital Q1046645)',
  '(capital Q3220821)',
  '(capital Q5035795)',
  '(capital Q425951)',
  '(capital Q16155537)',
  '(capital Q45182910)',
  '(capital Q21686048)',
  '(capital Q

In [19]:
dataDic = dict()
output_entites_list = []
input_entites_list = []

In [22]:
with open('./dataset/re-nlg_0-10000.json', 'r') as f:
    data = json.load(f)


In [ ]:

# generate the training data
count = 0
for d in data[0:10000]:
    clear_output(wait=True)
    count = count + 1
    print (count)
    if len(d['text'])<2500:     # that is text with less than 2500 len.
        
        input = expand(d['text'])
        output_entities = []
        for entities in d['entities']:
            if 'uri' in entities and 'http://www.wikidata.org/entity/' in entities['uri']:
                output_entities.append("("+entities['surfaceform'] + " " + entities['uri'].replace('http://www.wikidata.org/entity/','')+")")
        input_entites_list.append(input['input_entities'])
        output_entites_list.append(output_entities)        

In [4]:
# with open('input_entities.pickle', 'wb') as handle:
#     pickle.dump(input_entites_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('output_entities.pickle', 'wb') as handle:
#     pickle.dump(output_entites_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('output_entities.pickle', 'rb') as f:
    output_entites_list = pickle.load(f)

with open('input_entities.pickle', 'rb') as f:
    input_entites_list = pickle.load(f)

In [21]:
recalls = []
new_output_entities = []
count = 0
for i in range (0, len(input_entites_list)):
    setA = set(input_entites_list[i])
    setB = set(output_entites_list[i])
    if(len(setB)!=0):
        recall = len(setA.intersection(setB)) / len(setB)
        recalls.append(recall)
    if(len(setB) == 0):
        count +=1
        print(output_entites_list[i])

    new_output_entities.append(setA.intersection(setB))
print(count)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
213


In [30]:
with open('training_data.pickle', 'rb') as f:
    training_data = pickle.load(f)
print("Length ",len(training_data))

Length  9228


In [15]:
for i in range(0, len(training_data)):
    training_data[i]["summary"] = output_entites_list[i]

In [ ]:

with open('training_data_witoutfilter.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [17]:
sum(recalls)/len(recalls)

0.2597077596358534

In [27]:
#examples with no entities
no_Entites = [x for x in data[:10000] if len(x['entities']) <= 0 ]
print(len(no_Entites))
no_Entites[:1]


161


In [31]:
with open('training_data_witoutfilter.pickle', 'rb') as f:
    all_training_data = pickle.load(f)

In [34]:
len([x for x in all_training_data if len(x['summary']) <= 0])

213

In [33]:
[x for x in data if ('Transport in Angola comprises' in x["text"])]

[{'docid': 'http://www.wikidata.org/entity/Q3997810',
  'title': 'Transport in Angola',
  'text': 'Transport in Angola comprises:',
  'uri': 'http://www.wikidata.org/entity/Q3997810',
  'entities': [],
  'words_boundaries': [[0, 9], [10, 12], [13, 19], [20, 29], [29, 30]],
  'triples': [],
  'sentences_boundaries': [[0, 30]]}]

In [37]:
training_data_with_summary = [x for x in all_training_data if len(x['summary']) > 0]
len(training_data_with_summary)

9015

In [39]:
summaryFiltered = []
for example in training_data_with_summary:
    tempExample = example
    summary = []
    for entity in example["summary"]:
        if(entity in example["text"]):
            summary.append(entity)
    tempExample["summary"] = "".join(summary)
    summaryFiltered.append(tempExample)

In [40]:
with open('refiltered_training_data.pickle', 'wb') as handle:
    pickle.dump(summaryFiltered, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
with open('training_data_witoutfilter.pickle', 'rb') as f:
    training_data = pickle.load(f)
print("Length ",len(training_data))

Length  9228


In [7]:
recall_arry = []
for example in training_data:
    recall_nume = 0
    recall_denom = 0
    for entity in example["summary"]:
        if(entity.lower() in example["text"].lower()):
            recall_nume = recall_nume + 1
        recall_denom = recall_denom + 1
    recall_arry.append({'total': recall_denom, 'retreived':recall_nume })

In [14]:
recall_arry  = [x for x in recall_arry if( x["total"] > 0)]

In [15]:
from statistics import mean 

mean([x["retreived"]/x["total"] for x in recall_arry])

0.660843063932735

In [20]:
recall_arry = []
for example in training_data[0:10]:
    recall_nume = 0
    recall_denom = 0
    pprint.pprint(example["text"])
    for entity in example["summary"]:
        if(entity.lower() in example["text"].lower()):
            recall_nume = recall_nume + 1
        else:
            pprint.pprint(entity.lower())
        recall_denom = recall_denom + 1
    recall_arry.append({'total': recall_denom, 'retreived':recall_nume })

('The Austroasiatic languages, in recent classifications synonymous with '
 'Mon–Khmer, are a large language family of continental Southeast Asia, also '
 'scattered throughout India, Bangladesh, Nepal and the southern border of '
 'China. The name Austroasiatic comes from the Latin words for "south" and '
 '"Asia", hence "South Asia". Of these languages, only Vietnamese, Khmer, and '
 'Mon have a long-established recorded history, and only Vietnamese and Khmer '
 'have official status (in Vietnam and Cambodia, respectively). The rest of '
 'the languages are spoken by minority groups. Ethnologue identifies 168 '
 'Austroasiatic languages. These form thirteen established families (plus '
 'perhaps Shompen, which is poorly attested, as a fourteenth), which have '
 'traditionally been grouped into two, as Mon–Khmer and Munda. However, one '
 'recent classification posits three groups (Munda, Nuclear Mon-Khmer and '
 'Khasi-Khmuic) while another has abandoned Mon–Khmer as a taxon altogeth